In [10]:
import pandas as pd

import json


In [11]:
questions_raw = pd.read_csv('./data/questions_details_2.csv')

In [12]:
questions = questions_raw.copy()
# creator column is a list of strings. I want to create a new row for each creator
questions['creator'] = questions['creator'].apply(lambda x: eval(x) if isinstance(x, str) else x)
questions = questions.explode('creator')
questions = questions.drop_duplicates()



In [13]:
# use the document_date to create a new column based on YEAER-MONTH
questions['year-month'] = questions['document_date'].apply(lambda x: '-'.join(x.split('-')[:2]))
questions['member_id'] = questions['creator'].apply(lambda x: str(x).split('/')[-1] if 'person' in str(x) else None)

questions = questions[~questions['member_id'].isna()]

In [14]:
df_q_ym = questions.pivot_table(index='member_id', columns='year-month', values='id', aggfunc='size', fill_value=0)
df_q_ymd = questions.pivot_table(index='member_id', columns='document_date', values='id', aggfunc='size', fill_value=0)

# set index to be integer
df_q_ym.index = df_q_ym.index.astype(float)
df_q_ymd.index = df_q_ymd.index.astype(float)


df_q_ym.head()

year-month,2008-06,2009-01,2009-02,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,...,2024-03,2024-04,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02
member_id,,,,,,,,,,,,,,,,,,,,,
101039.0,0,0,0,0,0,0,0,0,0,0,...,0,5,0,1,0,2,0,1,3,1
101580.0,0,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
101585.0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,1
1023.0,0,0,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
102886.0,0,0,0,1,0,0,1,0,2,2,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_q_ym.to_csv('./data/silver/df_questions_by_period_YYYY-MM.csv')
df_q_ymd.to_csv('./data/silver/df_questions_by_period_YYYY-MM-DD.csv')